In [1]:
import boto
reload(boto)
from boto.mturk.price import Price
from boto.mturk.connection import MTurkConnection
import ujson as json
import time

from pymongo import MongoClient
import yaml

from tmilib import *

tmi_overrides['basedir'] = '/home/gkovacs/tmi-data/local_2016-04-06_03:43:54-07:00'

In [2]:
# made first payday on march 24 at 1:08am pacific time
# aka 1458806843.052402
# weeknum = 1

# made second payday on march 31 at 2:25am pacific time
# aka 1459416358.610529
#weeknum = 2

# made third payday on march 7 at 3:38am pacific time
# aka 1460025542.304254
weeknum = 3

run_test = True

In [3]:
#precompute_mturkid_to_time_last_active()

In [4]:
#worker_ids_with_history = set(json.load(open('list_of_worker_ids.json')))
mturkid_to_time_last_active = get_mturkid_to_time_last_active()

In [5]:
# active within the past 4 days
active_turkers = {x for x,y in mturkid_to_time_last_active.items() if y + 3600*24*3.5*1000 > 1000.0*time.time()}

In [6]:


secrets = yaml.load(open('.getsecret.yaml'))

client = MongoClient(secrets['mongourl'])

db = client[secrets['mongodb']]

bonus_collection = db.bonus

def havepaid(workerid, weeknum):
  item = bonus_collection.find_one(workerid)
  if item == None:
    return False
  if ('paidtime'+str(weeknum)) in item:
    return True
  return False

def recordbonus(workerid, weeknum, isactive):
  # weeknum: 1 to 5
  if weeknum not in [1, 2, 3, 4, 5]:
    print 'invalid value of weeknum ' + str(weeknum)
    return
  curtime = time.time()
  paidamount = 1.0
  if not isactive:
    paidamount = 0.05
  bonus_collection.update_one({'_id': workerid}, {'$set': {'numpaid': weeknum, 'lastpaid': curtime, 'active'+str(weeknum): isactive, 'paidtime'+str(weeknum): curtime, 'paidamount'+str(weeknum): paidamount}}, upsert=True)

def get_message_active(workerid, weeknum):
  return "Thank you for participating in our study! This $1 bonus is for your continued participation this week (week " + str(weeknum) + "). Please keep the extension installed, do not clear your browsing history, and continue using Chrome to browse the web to continue receiving this bonus. If you ever accidentally uninstall the extension or clear your browsing history, please go to https://tmi.stanford.edu/mturk3.html?username=" + workerid + " to reinstall it (you do not need to fill out the survey again)"

def get_message_inactive(workerid, weeknum):
  return "Thank you for participating in our study! You seem to have uninstalled our extension, cleared your browsing history, are no longer using Chrome to browse the web, or have not used the internet in the past 3 days, so we are unable to give you your full $1 bonus this week (week " + str(weeknum) + "). If you would like to receive the $1 bonus next week, please go to https://tmi.stanford.edu/mturk3.html?username=" + workerid + " to reinstall the extension (you just need to install the extension, you do not need to fill out the survey again)"

def get_message(workerid, weeknum, isactive):
  if isactive:
    return get_message_active(workerid, weeknum)
  return get_message_inactive(workerid, weeknum)

def pay_worker(workerid, weeknum, isactive):
  print 'paying worker ' + workerid + ' for week ' + str(weeknum) + ' isactive ' + str(isactive)
  if run_test:
    return
  message = get_message(workerid, weeknum, isactive)
  paidamount = 1.0
  if not isactive:
    paidamount = 0.05
  assignment_id = workerid_to_assignment[workerid]
  #print message
  mtc.grant_bonus(workerid, assignment_id, Price(paidamount), message)
  recordbonus(workerid, weeknum, isactive)

In [7]:
hit_ids = secrets['hit_ids']

mtc = MTurkConnection()

workerid_to_assignment = {}

for hit_id in hit_ids:
  for assignment in mtc.get_assignments(hit_id, page_size=100):
    assignment_id = assignment.AssignmentId
    worker_id = assignment.WorkerId
    workerid_to_assignment[worker_id] = assignment_id

In [8]:
#print len({x for x in workerid_to_assignment if x in active_turkers})

In [9]:
stats = Counter()
for worker_id,assignment_id in workerid_to_assignment.items():
  isactive = worker_id in active_turkers
  if not havepaid(worker_id, weeknum):
    pay_worker(worker_id, weeknum, isactive)
    if isactive:
      stats['active'] += 1
    else:
      stats['inactive'] += 1
  else:
    stats['already_paid'] += 1

In [10]:
print stats

Counter({'already_paid': 226})


In [11]:
#print len(workerid_to_assignment)
#print len({x for x in workerid_to_assignment if x in worker_ids_with_history})